In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM



In [2]:
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [150]:

# Define the ticker symbol
tickerSymbol = 'HMC'

# Get data on the ticker
tickerData = yf.Ticker(tickerSymbol)

# Get the historical prices for this ticker
tickerDf = tickerData.history(period='1d', start='2015-01-01', end='2023-04-28')

# Calculate the EMA with a window of 20 days
ema = tickerDf['Close'].ewm(span=20, adjust=False).mean()

# Merge the EMA with the original DataFrame
tickerDf = pd.concat([tickerDf, ema.rename('EMA')], axis=1)

# Define the feature and target variables
data = tickerDf[['Close', 'EMA']]
target = tickerDf['Close']

# Normalize the data using MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(data)



In [151]:
data.head()

,Close,EMA
Date,,
2015-01-02 00:00:00-05:00,24.785105,24.785105
2015-01-05 00:00:00-05:00,24.274158,24.736443
2015-01-06 00:00:00-05:00,24.140141,24.679652
2015-01-07 00:00:00-05:00,24.383049,24.651405
2015-01-08 00:00:00-05:00,24.692968,24.655363


In [152]:
# Define the number of time steps for the LSTM model
n_steps = 1000

# Create a function to split the data into input/output samples
def prepare_data(data, n_steps):
    X, y = [], []
    for i in range(len(data)):
        end_ix = i + n_steps
        if end_ix > len(data)-1:
            break
        seq_x, seq_y = data[i:end_ix, :], data[end_ix, 0]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

# Prepare the data for the LSTM model
X, y = prepare_data(data_scaled, n_steps)

# Split the data into training and testing sets
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

In [153]:
# Define the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(n_steps, 2)))
model.add(LSTM(units=50))
model.add(Dense(units=1))

In [154]:
# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

In [155]:
# Fit the model to the training data
model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)

In [156]:
# Evaluate the model on the testing data
mse = model.evaluate(X_test, y_test, verbose=0)
print('Mean Squared Error:', mse)

Mean Squared Error: 0.0008009092416614294


In [157]:
# Save the trained model to a file
model.save(f'{tickerSymbol.upper()}_model.h5')

In [158]:
# # # # Load the saved model from a file
# from keras.models import load_model
# loaded_model = load_model('1yr_aapl_lstm_model.h5')


In [159]:
from keras.models import load_model
loaded_model = load_model('models1000\\_model.h5')
# Define the ticker symbol
tickerSymbol = 'HMC'

# Get data on the ticker
tickerData = yf.Ticker(tickerSymbol)

# Get the historical prices for this ticker
tickerDf = tickerData.history(period='1d', start='2015-01-01', end='2023-04-28')

# Calculate the EMA with a window of 20 days
ema = tickerDf['Close'].ewm(span=20, adjust=False).mean()

# Merge the EMA with the original DataFrame
tickerDf = pd.concat([tickerDf, ema.rename('EMA')], axis=1)

# Define the feature and target variables
data = tickerDf[['Close', 'EMA']]
target = tickerDf['Close']

# Normalize the data using MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(data)

# Predict future prices
future_prices = []
last_n_steps = data_scaled[-n_steps:]
for i in range(90):  # predict 30 days in the future
    future_price = model.predict(last_n_steps.reshape(1, n_steps, 2))
    future_prices.append(future_price[0, 0])
    last_n_steps = np.append(last_n_steps[1:], [[future_price[0, 0], future_price[0, 0]]], axis=0)

# # Denormalize the predicted prices
# future_prices = scaler.inverse_transform(np.array(future_prices).reshape(-1, 1))
# Define the scaler object with the same feature range as before
scaler = MinMaxScaler(feature_range=(0, 1))

# Fit the scaler to the original target variable
scaler.fit(tickerDf['Close'].values.reshape(-1, 1))

# Denormalize the predicted prices using the scaler
future_prices = scaler.inverse_transform(np.array(future_prices).reshape(-1, 1))
# Print the predicted prices
print(future_prices)



[[26.449299]
 [26.591433]
 [26.69293 ]
 [26.795994]
 [26.906559]
 [27.02605 ]
 [27.154652]
 [27.291834]
 [27.436707]
 [27.588202]
 [27.745243]
 [27.906796]
 [28.07191 ]
 [28.239729]
 [28.409468]
 [28.58042 ]
 [28.751917]
 [28.923351]
 [29.094131]
 [29.263687]
 [29.431484]
 [29.596996]
 [29.759743]
 [29.919254]
 [30.075111]
 [30.226933]
 [30.374372]
 [30.517138]
 [30.654995]
 [30.787758]
 [30.915277]
 [31.037481]
 [31.154322]
 [31.2658  ]
 [31.371965]
 [31.472889]
 [31.568686]
 [31.659475]
 [31.745422]
 [31.82669 ]
 [31.903458]
 [31.975903]
 [32.04422 ]
 [32.108612]
 [32.16926 ]
 [32.22635 ]
 [32.28007 ]
 [32.330612]
 [32.37814 ]
 [32.42282 ]
 [32.46482 ]
 [32.504288]
 [32.54137 ]
 [32.57621 ]
 [32.608944]
 [32.63968 ]
 [32.66855 ]
 [32.695667]
 [32.72113 ]
 [32.74505 ]
 [32.767506]
 [32.78859 ]
 [32.808388]
 [32.82698 ]
 [32.844437]
 [32.860825]
 [32.876217]
 [32.890667]
 [32.904232]
 [32.916973]
 [32.928936]
 [32.940163]
 [32.95071 ]
 [32.96061 ]
 [32.969906]
 [32.97863 ]
 [32.98682 ]

In [160]:
[i for i in future_prices.flatten()]

[26.449299,
 26.591433,
 26.69293,
 26.795994,
 26.906559,
 27.02605,
 27.154652,
 27.291834,
 27.436707,
 27.588202,
 27.745243,
 27.906796,
 28.07191,
 28.239729,
 28.409468,
 28.58042,
 28.751917,
 28.923351,
 29.094131,
 29.263687,
 29.431484,
 29.596996,
 29.759743,
 29.919254,
 30.075111,
 30.226933,
 30.374372,
 30.517138,
 30.654995,
 30.787758,
 30.915277,
 31.037481,
 31.154322,
 31.2658,
 31.371965,
 31.472889,
 31.568686,
 31.659475,
 31.745422,
 31.82669,
 31.903458,
 31.975903,
 32.04422,
 32.108612,
 32.16926,
 32.22635,
 32.28007,
 32.330612,
 32.37814,
 32.42282,
 32.46482,
 32.504288,
 32.54137,
 32.57621,
 32.608944,
 32.63968,
 32.66855,
 32.695667,
 32.72113,
 32.74505,
 32.767506,
 32.78859,
 32.808388,
 32.82698,
 32.844437,
 32.860825,
 32.876217,
 32.890667,
 32.904232,
 32.916973,
 32.928936,
 32.940163,
 32.95071,
 32.96061,
 32.969906,
 32.97863,
 32.98682,
 32.994514,
 33.001743,
 33.00853,
 33.0149,
 33.02088,
 33.026493,
 33.03177,
 33.036716,
 33.04137,


In [161]:
import json

# Open the JSON file for reading
with open('tickers.json', 'r') as f:
    # Load the JSON data into a dictionary
    data = json.load(f)

# Now the data is a dictionary containing the contents of the JSON file
print(data)


{'tech': ['MSFT', 'NVDA', 'AAPL', 'AMZN', 'GOOGL', 'META'], 'pharma': ['AZN', 'RHHBY', 'NVS', 'BAYRY', 'SNY', 'PFE'], 'motor': ['TSLA', 'TM', 'MBGYY', 'F', 'GM', 'HMC']}
